In [1]:
from tensorflow.keras import datasets, layers, models

import cv2 as cv
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = (train_images / 255.0).astype(np.float32), test_images.astype(np.float32) / 255.0

In [29]:
def padImage(image, pixels=5):
    bottom = image[-pixels:]
    top = image[:pixels]

    img = np.insert(image, 0, bottom, 0)
    img = np.insert(img, len(img), top, 0)
    return img

In [32]:
# 23 is the ceiling of (16 * sqrt(2))
pixelsToPad = 6

X_train_polar = [cv.linearPolar(x, tuple(np.array(x.shape[:2])/2), 23, cv.WARP_FILL_OUTLIERS) for x in train_images]
X_train_polar = [padImage(x, pixels=pixelsToPad) for x in X_train_polar]
X_train_polar = np.array(X_train_polar)

X_test_polar = [cv.linearPolar(x, tuple(np.array(x.shape[:2])/2), 23, cv.WARP_FILL_OUTLIERS) for x in test_images]
X_test_polar = [padImage(x, pixels=pixelsToPad) for x in X_test_polar]
X_test_polar = np.array(X_test_polar)

# Rotate test set
X_test_r_polar = [tfa.image.rotate(x, np.random.uniform(-np.pi/2., np.pi/2.)).numpy() for x in test_images]
X_test_r_polar = [cv.linearPolar(x, tuple(np.array(x.shape[:2])/2), 23, cv.WARP_FILL_OUTLIERS) for x in X_test_r_polar]
X_test_r_polar = [padImage(x, pixels=pixelsToPad) for x in X_test_r_polar]
X_test_r_polar = np.array(X_test_r_polar)

In [33]:
model = models.Sequential()
model.add(layers.Input(shape=X_train_polar.shape[1:]))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

model.add(layers.GlobalMaxPooling2D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 42, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 4, 128)         73856     
_________________________________________________________________
global_max_pooling2d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)               

In [40]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_polar, train_labels, batch_size=32, epochs=3)

Epoch 1/3
1563/1563 [==============================] - 35s 22ms/step - loss: 1.0244 - accuracy: 0.6365
Epoch 2/3
1563/1563 [==============================] - 35s 22ms/step - loss: 0.9660 - accuracy: 0.6562
Epoch 3/3
1563/1563 [==============================] - 65s 41ms/step - loss: 0.9040 - accuracy: 0.6803


In [42]:
model.evaluate(X_test_polar, test_labels, verbose=2)

# Plain accuracy
#accuracy = np.mean(model.predict(X_test_polar).argmax(axis=1) == test_labels)*100
# "Rotated" accuracy
#accuracy_rotated = np.mean(model.predict(X_test_r_polar).argmax(axis=1) == test_labels)*100

#print(accuracy, accuracy_rotated)

313/313 - 4s - loss: 1.0595 - accuracy: 0.6215


[1.0595182180404663, 0.6215000152587891]

In [43]:
model.evaluate(X_test_r_polar, test_labels, verbose=2)

313/313 - 3s - loss: 1.7254 - accuracy: 0.4255


[1.72537362575531, 0.4255000054836273]